In [1]:
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
urls_page = BeautifulSoup(requests \
            .get('http://rosettacode.org/wiki/Category:Solutions_by_Programming_Task') \
            .content, 'html.parser').find_all('div', {'class' : 'mw-content-ltr'})[-1]

In [3]:
#urls_page.find('div', {'class' : 'mw-content-ltr'})
urls = re.findall(r'href="(.*)" ', str(urls_page))#[:35]
pages = [BeautifulSoup(requests.get('http://rosettacode.org'+url) \
                       .content, 'html.parser') for url in urls]

In [4]:
#doors = requests.get('http://rosettacode.org/wiki/100_doors')

# doors = BeautifulSoup(requests.get('http://rosettacode.org/wiki/100_doors') \
#                       .content, 'html.parser')



In [5]:
# doors.prettify

In [6]:
langs = ['c', 'csharp', 'lisp', 'clojure', 'haskell', 
         'java', 'javascript', 'ocaml', 'perl', 'php', 
         'python', 'ruby', 'scala', 'scheme']

# lang_dict = {lang : '' for lang in langs}
lang_X = []
lang_y = []

for page in pages:
    for string in page.find_all('pre'):
        for lang in langs:
            if re.search(r'"'+ lang + ' hi', str(string)):
                # lang_dict[lang] = lang_dict[lang] + string.get_text()
                lang_X.append(string.get_text())
                lang_y.append(lang)
#print(lang_dict)

In [7]:
lang_X

['(defun rep (n x)   (if (zp n)       nil       (cons x             (rep (- n 1) x))))\xa0(defun toggle-every-r (n i bs)   (if (endp bs)       nil       (cons (if (zp i)                 (not (first bs))                 (first bs))             (toggle-every-r n (mod (1- i) n) (rest bs)))))\xa0(defun toggle-every (n bs)   (toggle-every-r n (1- n) bs))\xa0(defun 100-doors (i doors)   (if (zp i)       doors       (100-doors (1- i) (toggle-every i doors))))',
 '#include <stdio.h>\xa0int main(){  char is_open[100] = { 0 };  int pass, door;\xa0  /* do the 100 passes */  for (pass = 0; pass < 100; ++pass)    for (door = pass; door < 100; door += pass+1)      is_open[door] = !is_open[door];\xa0  /* output the result */  for (door = 0; door < 100; ++door)    printf("door #%d is\xa0%s.\\n", door+1, (is_open[door]? "open" : "closed"));\xa0  return 0;}',
 '#include <stdio.h>\xa0#define NUM_DOORS 100\xa0int main(int argc, char *argv[]){  int is_open[NUM_DOORS] = { 0 } ;  int * doorptr, * doorlimit =

In [8]:
class FunctionFeaturizer(TransformerMixin):
    def __init__(self, *featurizers):
        self.featurizers = featurizers
        
    def fit(self, X, y=None):
        '''Returns self after transformations'''
        return self
    
    def transform(self, X):
        '''Given a list of original data, return a list of feature vectors'''
        feature_vectors = []
        for x in X:
            feature_vector = [f(x) for f in self.featurizers]
            feature_vectors.append(feature_vector)
        return np.array(feature_vectors)

In [405]:
# chars = ['(', ')', '{', '}', '[', ']', '<', '>', '$', '/', ' ', '\t']
# frequencies = [lambda text: text.count(char)/len(text) for char in chars]
def paren_freq(text):
    '''checks the frequency of the ( character'''
    return text.count('(')/len(text)

def thesis_freq(text):
    '''checks the frequency of the ) character'''
    return text.count(')')/len(text)
def paren_colon(text):
    return text.count('(:')/len(text)
def paren_plus(text):
    return text.count('(+')/len(text)
def brac_freq(text):
    '''checks the frequency of the ) character'''
    return text.count('[')/len(text)
def carrot_freq(text):
    return text.count('^')/len(text)
def ket_freq(text):
    '''checks the frequency of the ) character'''
    return text.count(']')/len(text)
def brackets_freq(text):
    return text.count('{}')/len(text)
# print(*frequencies)
def lesser_freq(text):
    '''checks the frequency of the ) character'''
    return text.count('<')/len(text)
def greater_freq(text):
    '''checks the frequency of the ) character'''
    return text.count('>')/len(text)
def slash_freq(text):
    '''checks the frequency of the ) character'''
    return text.count('/')/len(text)
def dollar_freq(text):
    '''checks the frequency of the ) character'''
    return text.count('$')/len(text)
def double_quote_freq(text):
    '''checks the frequency of the ) character'''
    return text.count('"')/len(text)
def space_freq(text):
    '''checks the frequency of the ) character'''
    return text.count(' ')/len(text)
def semi_freq(text):
    '''checks the frequency of the ) character'''
    return text.count(';')/len(text)
def colon_freq(text):
    '''checks the frequency of the ) character'''
    return text.count(':')/len(text)


def at_freq(text):
    '''checks the frequency of the ) character'''
    return text.count('@')/len(text)
def star_freq(text):
    '''checks the frequency of the ) character'''
    return text.count('*')/len(text)
def pipe_freq(text):
    return text.count('|')/len(text)
def pipe_2_freq(text):
    return text.count('||')/len(text)
def and_2_freq(text):
    return text.count('&&')/len(text)
def __freq(text):
    return text.count('_')/len(text)
def hyph_freq(text):
    return text.count('-')/len(text)
def dundr_freq(text):
    return text.count('__')/len(text)
def arrow_freq(text):
    return text.count('->')/len(text)
def arrow_2_freq(text):
    return text.count('=>')/len(text)
def slash_splat_freq(text):
    return text.count('/*')/len(text)
def quote_paren_freq(text):
    return text.count("'(")/len(text)
def space_dot_freq(text):
    return text.count(' . ')/len(text)
def slash_2_freq(text):
    return text.count('//')/len(text)
def hyph_2_freq(text):
    return text.count('--')/len(text)
def curl_hyph_freq(text):
    return text.count('{-')/len(text)

def from_freq(text):
    return text.count('from')/len(text)
def let_freq(text):
    return text.count('let')/len(text)
def val_freq(text):
    return text.count('val')/len(text)
def class_freq(text):
    return text.count('class')/len(text)
def subst_freq(text):
    return text.count('subst')/len(text)
def new_freq(text):
    return text.count('new')/len(text)
def old_freq(text):
    return text.count('old')/len(text)
def p_extend_freq(text):
    return text.count('(extend')/len(text)
def p_extend_hyph_freq(text):
    return text.count('(extend-')/len(text)
def defn_freq(text):
    return text.count('defn')/len(text)
def println_freq(text):
    return text.count('println')/len(text)

def import_freq(text):
    return text.count('import')/len(text)

def call_freq(text):
    return text.count('call')/len(text)

def null_freq(text):
    return text.count('null')/len(text)

def NIL_freq(text):
    return text.count('NIL')/len(text)
def using_freq(text):
    return text.count('using')/len(text)
def static_freq(text):
    return text.count('static')/len(text)
def Console_freq(text):
    return text.count('Console')/len(text)
def void_freq(text):
    return text.count('void')/len(text)
def module_freq(text):
    return text.count('module')/len(text)
def printp_freq(text):
    return text.count('print(')/len(text)
def elsif_freq(text):
    return text.count('elsif')/len(text)

def where_freq(text):
    return text.count('where')/len(text)
def var_freq(text):
    return text.count('var')/len(text)
    
def function_freq(text):
    return text.count('function')/len(text)

def end_freq(text):
    return text.count('end')/len(text)
    
def case_freq(text):
    return text.count('case')/len(text)

def type_freq(text):
    return text.count('type')/len(text)

def object_freq(text):
    return text.count('object')/len(text)
def private_freq(text):
    return text.count('private')/len(text)
def public_freq(text):
    return text.count('public')/len(text)
def def_freq(text):
    return text.count('def ')/len(text)
def implicit_freq(text):
    return text.count('implicit')/len(text)
def final_freq(text):
    return text.count('final')/len(text)
def proc_freq(text):
    return text.count('proc')/len(text)
def set_freq(text):
    return text.count('set')/len(text)
def return_freq(text):
    return text.count('return')/len(text)
def if_freq(text):
    return text.count('if')/len(text)
def dict_freq(text):
    return text.count('dict')/len(text)
def extend_freq(text):
    return text.count('extend')/len(text)
def colon_2_freq(text):
    return text.count('::')/len(text)
def require_freq(text):
    return text.count('require')/len(text)
def String_freq(text):
    return text.count('String')/len(text)
def defun_freq(text):
    return text.count('defun')/len(text)
def eql_freq(text):
    return text.count('eql')/len(text)
def boolean_freq(text):
    return text.count('boolean')/len(text)
def List_freq(text):
    return text.count('List')/len(text)
def package_freq(text):
    return text.count('package')/len(text)
def paren_define_freq(text):
    return text.count('(define')/len(text)
def paren_lambda_freq(text):
    return text.count('(lambda')/len(text)
def paren_con_freq(text):
    return text.count('(con')/len(text)
def intero_null_freq(text):
    return text.count('null?')/len(text)
def my_freq(text):
    return text.count('my')/len(text)
def setf_freq(text):
    return text.count('setf')/len(text)
def nth_freq(text):
    return text.count('nth')/len(text)
def interface_freq(text):
    return text.count('interface')/len(text)
def dollar_this_freq(text):
    return text.count('$this')/len(text)
def data_freq(text):
    return text.count('data')/len(text)
def deriving_freq(text):
    return text.count('deriving')/len(text)
def foldr_freq(text):
    return text.count('foldr')/len(text)
def printf_freq(text):
    return text.count('printf')/len(text)
def pound_include_freq(text):
    return text.count('#include')/len(text)
def Any_freq(text):
    return text.count('Any')/len(text)
def stdioh_freq(text):
    return text.count('stdio.h')/len(text)
def paren_and_freq(text):
    return text.count('(and')/len(text)
def namespace_freq(text):
    return text.count('namespace')/len(text)
def int_main_freq(text):
    return text.count('int main')/len(text)
def dotimes_freq(text):
    return text.count('dotimes')/len(text)
def dolist_freq(text):
    return text.count('dolist')/len(text)
def makelist_freq(text):
    return text.count('make-list')/len(text)
def mapcar_freq(text):
    return text.count('mapcar')/len(text)

def avg_word_len(text):
    words = re.split('\W*', text)
    return sum([len(word) for word in words])/len(words)

def percentage_of_punctuation(text):
    total_length = len(text)
    text = re.sub(r'[\w\s]', '', text)
    punc_length = len(text)
    
    return punc_length / total_length

def case_ratio(text):
    try:
        up = text.count(r'A-Z')
        down = text.count(r'a-z')
        return up/down
    except:
        return 0
    
def camel_case(text):
    if re.search(r'[ .]+[a-z]*[A-Z]*[a-z]*', text):
        return True
    else:
        return False


In [406]:
features = FunctionFeaturizer(paren_freq, thesis_freq, brac_freq,
                              ket_freq, lesser_freq, greater_freq,
                              slash_freq, dollar_freq, double_quote_freq,
                              space_freq, semi_freq, colon_freq, defn_freq,
                              import_freq, at_freq, star_freq, avg_word_len,
                              percentage_of_punctuation, null_freq, NIL_freq,
                              void_freq, var_freq, function_freq, end_freq,
                              pipe_freq, __freq, hyph_freq, dundr_freq,
                              case_ratio, case_freq, val_freq, type_freq,
                              object_freq, private_freq, public_freq,
                              def_freq, implicit_freq, final_freq, proc_freq,
                              set_freq, carrot_freq, if_freq, dict_freq,
                              arrow_freq, module_freq, extend_freq,
                              colon_2_freq, arrow_2_freq, require_freq,
                              from_freq, printp_freq, brackets_freq,
                              call_freq, p_extend_freq, paren_colon,
                              paren_define_freq, paren_plus, paren_lambda_freq,
                              String_freq, boolean_freq, slash_splat_freq,
                              let_freq, paren_con_freq, quote_paren_freq,
                              where_freq, space_dot_freq, pipe_2_freq,
                              and_2_freq, camel_case, p_extend_hyph_freq,
                              package_freq, List_freq, slash_2_freq,
                              subst_freq, new_freq, old_freq, using_freq,
                              static_freq, Console_freq, defun_freq, 
                              eql_freq, hyph_2_freq, curl_hyph_freq,
                              intero_null_freq, setf_freq, nth_freq,
                              dollar_this_freq, data_freq, deriving_freq,
                              foldr_freq, printf_freq, pound_include_freq,
                              Any_freq, stdioh_freq, paren_and_freq, 
                              elsif_freq, namespace_freq, interface_freq,
                              int_main_freq, println_freq, dotimes_freq,
                              dolist_freq, makelist_freq, mapcar_freq)


In [446]:
lang_X_train, lang_X_test, lang_y_train, lang_y_test = train_test_split(lang_X, lang_y)

In [447]:
pipe = make_pipeline(features, DecisionTreeClassifier())
pipe.fit(lang_X_train, lang_y_train)
pipe.score(lang_X_train, lang_y_train)

0.99929103154909604

In [448]:
print(pipe.score(lang_X_test, lang_y_test))
print(classification_report(pipe.predict(lang_X_test), lang_y_test))

0.77045696068
             precision    recall  f1-score   support

          c       0.87      0.85      0.86        88
    clojure       0.51      0.58      0.54        38
     csharp       0.67      0.69      0.68        26
    haskell       0.86      0.83      0.84       100
       java       0.93      0.91      0.92        56
 javascript       0.64      0.82      0.72        44
       lisp       0.74      0.68      0.71        99
      ocaml       0.79      0.74      0.76        50
       perl       0.86      0.82      0.84        88
        php       0.63      0.76      0.69        29
     python       0.83      0.82      0.83       112
       ruby       0.75      0.74      0.74       124
      scala       0.67      0.68      0.68        57
     scheme       0.66      0.63      0.64        30

avg / total       0.78      0.77      0.77       941



In [449]:
with open('test.csv') as test_csv:
    from csv import reader
    test_files = reader(test_csv)
    right = 0
    for line in test_files:
        with open('test/'+line[0]+'.txt') as test:
            pred = pipe.predict([test.read()])
            if pred == line[1]:
                right += 1
            else:
                print(pred, line[1])
    print(right/32)
        

['lisp'] clojure
['ruby'] javascript
['haskell'] scheme
['ocaml'] java
['csharp'] java
['haskell'] tcl
['haskell'] tcl
['perl'] php
0.75


In [470]:
for x in range(len(lang_X)):
    if lang_X[x].count('end  end'):
        print(lang_y[x], end=' ')

ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby ruby 

In [452]:
'c' - 'stdio.h'
    'csharp' - 'namespace', 'void '
    'java' - 'void '

'lisp' - '(defun', 'let '
'haskell' - 
'perl'
'php', 
'python'
'ruby' - r'end *end'
'javascript' - 'var '
'scala' - 'var ', 'let '
'clojure' - 'defn', 'let'
    'ocaml' - 'let'
    'scheme' - 'let'

<function sklearn.pipeline.Pipeline.predict>